# Visualizer for Auto‑Proofread Neurons with Neuroglancer

This notebook imports the **NEURD-Neuroglancer-Generator** module and generates Neuroglancer state for specified rows in the H01 Autoproof table.

It performs the following:
1. Loads configuration from `config.yaml` (paths, dataset, bucket, layer names - parametrized for H01 by default)
2. Connects to the **connects_neuvue** API to fetch the Autoproof table
3. Lets you pick a row (e.g., by `segment_id`), then:
   - Pulls the proofread mesh + skeletons
   - Uploads precomputed fragments to S3
   - Builds and returns a Neuroglancer share link centered on that proofread object.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import connects_neuvue
from connects_neuvue.utils import aws_utils as aws
from connects_neuvue.h01_c2 import api
from generate_ng_state import generate_ng_state

/Users/paniga1/connects-neuvue/lib/python3.13/site-packages/datajoint/plugin.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/paniga1/connects_neuvue/connects_neuvue/h01_c2/api.py:27: UserWarning: neurd.neuron_visualizations not found, some visualizations will not work. Please install the neurd package to enable visualizations
  warn("neurd.neuron_visualizations not found, some visualizations will not work. "


In [3]:
secret_dict = aws.get_secret()
fetcher = api.API(secret_dict=secret_dict)

[2025-09-05 03:12:43,513][INFO]: DataJoint 0.14.5 connected to admin@neurd-datajoint.cluster-cjc6cqmcqirl.us-east-1.rds.amazonaws.com:3306
INFO:datajoint:DataJoint 0.14.5 connected to admin@neurd-datajoint.cluster-cjc6cqmcqirl.us-east-1.rds.amazonaws.com:3306


### Extract & Query the Autoproofread Neuron Table

#### Viewing autoproofread datajoint table attributes and data types

In [4]:
print(fetcher.autoproof_table().describe())

-> connects_neuvue.h01_c2.schema.DecompositionCellType
---
multiplicity         : tinyint unsigned             # the total number of neurons that came from the parent segment id
cell_type_used=null  : enum('external','baylor')    
cell_type=null       : enum('excitatory','inhibitory','other','unknown') 
nucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'.
nuclei_distance      : double                       # the distance to the closest nuclei (even if no matching nuclei found)
n_nuclei_in_radius   : tinyint unsigned             # the number of nuclei within the search radius of 15000 belonging to that segment
n_nuclei_in_bbox     : tinyint unsigned             # the number of nuclei within the bounding box of that soma
centroid_x=null      : int unsigned                 # (EM voxels)
centroid_y=null      : int unsigned                 # (EM voxels)
centroid_z=null      : int unsigned                 # (EM voxels)
cen

In [5]:
help(fetcher.autoproof_table)

Help on class AutoProofreadNeuron in module connects_neuvue.h01_c2.schema:

class AutoProofreadNeuron(datajoint.user_tables.Computed)
 |  Table definition:
 |
 |
 |      -> h01.DecompositionCellType
 |      ---
 |
 |  Method resolution order:
 |      AutoProofreadNeuron
 |      datajoint.user_tables.Computed
 |      datajoint.user_tables.UserTable
 |      datajoint.table.Table
 |      datajoint.expression.QueryExpression
 |      datajoint.autopopulate.AutoPopulate
 |      builtins.object
 |
 |  Data and other attributes defined here:
 |
 |  Obj = <class 'connects_neuvue.h01_c2.schema.AutoProofreadNeuron.Obj'>
 |      Table definition:
 |
 |
 |              ->master
 |              ---
 |              neuron_mesh_faces: <auto_proof_meshes_dtype>     # face indices for neuron_mesh_faces of final proofread neuron
 |              neuron_skeleton: <auto_proof_skeletons_dtype>     # skeleton array for neuron_skeleton of final proofread neuron
 |
 |
 |              dendrite_mesh_faces: <auto_

#### Query & filter autoproof table

To make it easier to query, let us select which columns in particular you want to see. We will visualize the autoproof datajoint table with solely its cell_type, baylor_cell_type_after_proof, and axon_skeletal_length columns. Note, by default, it includes segment_id, decimation_ratio, and split_index in the filtered table. 

In [6]:
metadata_columns = [
    "axon_skeletal_length",
    "cell_type",
    "baylor_cell_type_after_proof",
]
queried_cell_type_axonal_length = fetcher.autoproof_table().proj(*metadata_columns)

In [7]:
queried_cell_type_axonal_length

segment_id ID for segment in this materialization,decimation_ratio,split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,cell_type,baylor_cell_type_after_proof,axon_skeletal_length
315456094,0.18,0,inhibitory,inhibitory,104148
316084337,0.18,0,inhibitory,inhibitory,46435
329165083,0.18,0,excitatory,excitatory,0
329194148,0.18,0,inhibitory,inhibitory,10522
329209033,0.18,0,excitatory,excitatory,36677
329458855,0.18,0,excitatory,excitatory,50720
329517693,0.18,0,excitatory,inhibitory,27027
330158986,0.18,0,inhibitory,inhibitory,217701
343415521,0.18,0,excitatory,excitatory,0
343606869,0.18,0,inhibitory,inhibitory,13550


Next, let us extract the specific autoproof neurons we want to visualize in neuroglancer, by quering the datajoint table. 

For example, if we want to run the NEURD-Neuroglancer-Generator module specifically on neurons thresholded on axonal skeletal length where the cell type was switched from excitatory to inhibitory post-autoproofreading, we can query the autoproof table as follows:

In [8]:
filter_length_proof_cell_type = "(axon_skeletal_length > 500000 and cell_type = 'excitatory' and baylor_cell_type_after_proof='inhibitory')"
filtered_cell_type_axonal_length = queried_cell_type_axonal_length & filter_length_proof_cell_type

In [9]:
filtered_cell_type_axonal_length

segment_id ID for segment in this materialization,decimation_ratio,split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,cell_type,baylor_cell_type_after_proof,axon_skeletal_length
2688016370,0.18,0,excitatory,inhibitory,1007654
2948993175,0.18,0,excitatory,inhibitory,1305923
5293286102,0.18,0,excitatory,inhibitory,774544
30508719663,0.18,0,excitatory,inhibitory,557935
42183852581,0.18,0,excitatory,inhibitory,923705


And because the NEURD-Neuroglancer-Generator module only needs the segment_ids to visualize the queried neurons, let us extract these IDs as a list:

In [10]:
segment_ids = filtered_cell_type_axonal_length.fetch("segment_id").tolist()

In [11]:
segment_ids

[2688016370, 2948993175, 5293286102, 30508719663, 42183852581]

### Run NEURD-Neuroglancer-Generator on the selected neuron IDs

For this example, we will use the segment_ids that we extracted by filtering the autoproof table in the example case above, but you can also simply define a list **segment_ids=[]** with the IDs you wish to visualize.

In [12]:
ng_url = generate_ng_state(segment_ids) 
print(ng_url)

Using default Google credentials. There is no ~/.cloudvolume/secrets/google-secret.json set.

        google-cloud-python might not find your credentials. Your credentials
        might be located in $HOME/.config/gcloud/legacy_credentials/$YOUR_GMAIL/adc.json

        If they are you can export your credentials like so:
        export GOOGLE_APPLICATION_CREDENTIALS="$HOME/.config/gcloud/legacy_credentials/$YOUR_GMAIL/adc.json"
      


/Users/paniga1/connects-neuvue/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


https://spelunker.cave-explorer.org/#!middleauth+https://global.brain-wire-test.org//nglstate/api/v1/6287335574470656
